## 뉴스 크롤링 후 저장 

In [16]:
import sys, os
import requests
import selenium
from selenium import webdriver
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime
import pickle, json, glob, time
from tqdm import tqdm



###### 날짜 저장 ##########
date = str(datetime.now())
date = date[:date.rfind(':')].replace(' ', '_')
date = date.replace(':','시') + '분'

sleep_sec = 0.5


####### 언론사별 본문 위치 태그 파싱 함수 ###########
print('본문 크롤링에 필요한 함수를 로딩하고 있습니다...\n' + '-' * 100)
def crawling_main_text(url):

    req = requests.get(url)
    req.encoding = None
    soup = BeautifulSoup(req.text, 'html.parser')
    
    # 연합뉴스
    if ('://yna' in url) | ('app.yonhapnews' in url): 
        main_article = soup.find('article', {'class':'story-news article'})
        if main_article == None:
            main_article = soup.find('div', {'class' : 'article-txt'})
            
        text = main_article.text
        
    # MBC 
    elif '//imnews.imbc' in url: 
        text = soup.find('div', {'itemprop' : 'articleBody'}).text
        
    # 매일경제(미라클), req.encoding = None 설정 필요
    elif 'mirakle.mk' in url:
        text = soup.find('div', {'class' : 'view_txt'}).text
        
    # 매일경제, req.encoding = None 설정 필요
    elif 'mk.co' in url:
        text = soup.find('div', {'class' : 'art_txt'}).text
        
    # SBS
    elif 'news.sbs' in url:
        text = soup.find('div', {'itemprop' : 'articleBody'}).text
    
    # KBS
    elif 'news.kbs' in url:
        text = soup.find('div', {'id' : 'cont_newstext'}).text
        
    # JTBC
    elif 'news.jtbc' in url:
        text = soup.find('div', {'class' : 'article_content'}).text
        
    # 그 외
    else:
        text == None
        
    return text.replace('\n','').replace('\r','').replace('<br>','').replace('\t','')
    
    
press_nm = 'MBC'

print('검색할 언론사 : {}'.format(press_nm))


############### 브라우저를 켜고 검색 키워드 입력 ####################
query = input('검색할 키워드  : ')
news_num = int(input('수집 뉴스의 수(숫자만 입력) : '))

print('\n' + '=' * 100 + '\n')

print('브라우저를 실행시킵니다(자동 제어)\n')
chrome_path = 'C:/Users/jhjh3/chromedriver_win32 (1)/chromedriver.exe'
browser = webdriver.Chrome(chrome_path)

news_url = 'https://search.naver.com/search.naver?where=news&query={}'.format(query)
browser.get(news_url)
time.sleep(sleep_sec)


######### 언론사 선택 및 confirm #####################
print('설정한 언론사를 선택합니다.\n')

search_opn_btn = browser.find_element_by_xpath('//a[@class="btn_option _search_option_open_btn"]')
search_opn_btn.click()
time.sleep(sleep_sec)

bx_press = browser.find_element_by_xpath('//div[@role="listbox" and @class="api_group_option_sort _search_option_detail_wrap"]//li[@class="bx press"]')

# 기준 두번 째(언론사 분류순) 클릭하고 오픈하기
press_tablist = bx_press.find_elements_by_xpath('.//div[@role="tablist" and @class="option"]/a')
press_tablist[1].click()
time.sleep(sleep_sec)

# 첫 번째 것(언론사 분류선택)
bx_group = bx_press.find_elements_by_xpath('.//div[@class="api_select_option type_group _category_select_layer"]/div[@class="select_wrap _root"]')[0]

press_kind_bx = bx_group.find_elements_by_xpath('.//div[@class="group_select _list_root"]')[0]
press_kind_btn_list = press_kind_bx.find_elements_by_xpath('.//ul[@role="tablist" and @class="lst_item _ul"]/li/a')


for press_kind_btn in press_kind_btn_list:
    
    # 언론사 종류를 순차적으로 클릭(좌측)
    press_kind_btn.click()
    time.sleep(sleep_sec)
    
    # 언론사선택(우측)
    press_slct_bx = bx_group.find_elements_by_xpath('.//div[@class="group_select _list_root"]')[1]
    # 언론사 선택할 수 있는 클릭 버튼
    press_slct_btn_list = press_slct_bx.find_elements_by_xpath('.//ul[@role="tablist" and @class="lst_item _ul"]/li/a')
    # 언론사 이름들 추출
    press_slct_btn_list_nm = [psl.text for psl in press_slct_btn_list]
    
    # 언론사 이름 : 언론사 클릭 버튼 인 딕셔너리 생성
    press_slct_btn_dict = dict(zip(press_slct_btn_list_nm, press_slct_btn_list))
    
    # 원하는 언론사가 해당 이름 안에 있는 경우
    # 1) 클릭하고
    # 2) 더이상 언론사분류선택 탐색 중지
    if press_nm in press_slct_btn_dict.keys():
        print('<{}> 카테고리에서 <{}>를 찾았으므로 탐색을 종료합니다'.format(press_kind_btn.text, press_nm))
        
        press_slct_btn_dict[press_nm].click()
        time.sleep(sleep_sec)
        
        break



################ 뉴스 크롤링 ########################

print('\n크롤링을 시작합니다.')
# ####동적 제어로 페이지 넘어가며 크롤링
news_dict = {}
idx = 1
cur_page = 1

pbar = tqdm(total=news_num ,leave = True)
    
while idx < news_num:

    table = browser.find_element_by_xpath('//ul[@class="list_news"]')
    li_list = table.find_elements_by_xpath('./li[contains(@id, "sp_nws")]')
    area_list = [li.find_element_by_xpath('.//div[@class="news_area"]') for li in li_list]
    a_list = [area.find_element_by_xpath('.//a[@class="news_tit"]') for area in area_list]
 
    for n in a_list[:min(len(a_list), news_num-idx+1)]:
        n_url = n.get_attribute('href')
        news_dict[idx] = {'title' : n.get_attribute('title'), 
                          'url' : n_url,
                          'text' : crawling_main_text(n_url)}
        
        idx += 1
        pbar.update(1)
        
    if idx < news_num:
        cur_page +=1

        pages = browser.find_element_by_xpath('//div[@class="sc_page_inner"]')
        next_page_url = [p for p in pages.find_elements_by_xpath('.//a') if p.text == str(cur_page)][0].get_attribute('href')

        browser.get(next_page_url)
        time.sleep(sleep_sec)
    else:
        pbar.close()
        
        print('\n브라우저를 종료합니다.\n' + '=' * 100)
        time.sleep(0.7)
        browser.close()
        break

#### 데이터 전처리하기 ###################################################### 

print('데이터프레임 변환\n')
news_df = DataFrame(news_dict).T

folder_path = os.getcwd()
xlsx_file_name = '네이버뉴스_본문_{}개_{}_{}.xlsx'.format(news_num, query, date)

news_df.to_excel(xlsx_file_name)

print('엑셀 저장 완료 | 경로 : {}\\{}\n'.format(folder_path, xlsx_file_name))

os.startfile(folder_path)

print('=' * 100 + '\n결과물의 일부')
news_df

본문 크롤링에 필요한 함수를 로딩하고 있습니다...
----------------------------------------------------------------------------------------------------
검색할 언론사 : MBC
검색할 키워드  : 코로나
수집 뉴스의 수(숫자만 입력) : 3


브라우저를 실행시킵니다(자동 제어)



C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_23372/1155766905.py:83: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(chrome_path)


설정한 언론사를 선택합니다.



C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_23372/1155766905.py:93: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_opn_btn = browser.find_element_by_xpath('//a[@class="btn_option _search_option_open_btn"]')
C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_23372/1155766905.py:97: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  bx_press = browser.find_element_by_xpath('//div[@role="listbox" and @class="api_group_option_sort _search_option_detail_wrap"]//li[@class="bx press"]')


<방송/통신> 카테고리에서 <MBC>를 찾았으므로 탐색을 종료합니다

크롤링을 시작합니다.


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_23372/1155766905.py:152: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  table = browser.find_element_by_xpath('//ul[@class="list_news"]')
C:\Users\jhjh3\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:393: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.58it/s]



브라우저를 종료합니다.
데이터프레임 변환

엑셀 저장 완료 | 경로 : C:\Users\jhjh3\PJT_02\네이버뉴스_본문_3개_코로나_2022-03-23_17시31분.xlsx

결과물의 일부


,title,url,text
1,"김 총리 ""1~2주가 코로나 위기극복 전환점‥중증·사망 최소화에 총력""",https://imnews.imbc.com/news/2022/politics/art...,"사진 제공:연합뉴스김부겸 국무총리는 코로나 확산세와 관련해 ""앞으로 1~2주간이 코..."
2,먹는 코로나약 '라게브리오' 성인 대상 긴급승인‥국내 두 번째,https://imnews.imbc.com/news/2022/society/arti...,자료 제공: 연합뉴스화이자의 코로나19 먹는 치료제 '팍스로비드'에 이어 MSD의 ...
3,"코로나 사망자 증가세 ""간접사망 포함시 누적 3만명 추정""",https://imnews.imbc.com/news/2022/society/arti...,자료 제공: 연합뉴스코로나19로 인한 사망자가 증가하고 있는 가운데 코로나와 연관이...


## 뉴스 구분 넣고 요약하고 음성파일 만들기

In [46]:
len(news_df.title)
# news_list = []

import os
import sys
import requests
import json
import urllib.request

client_id = "qj40wl5n0t"
client_secret = "1xzXEYgwlsbv9RQxd1onUSByiyvtBDPFV9NEY1mv"

for i in range(len(news_df.title)):
    headers = {
    "X-NCP-APIGW-API-KEY-ID": client_id,
    "X-NCP-APIGW-API-KEY": client_secret,
    "Content-Type": "application/json"
}
    language = "ko" # Language of document (ko, ja )
    model = "news" # Model used for summaries (general, news)
    tone = "2" # Converts the tone of the summarized result. (0, 1, 2, 3)
    summaryCount = "3" # This is the number of sentences for the summarized document.
    url= "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize" 
    title= ""
    content = news_df.text[i+1]
    data = {
        "document": {
        "title": title,
        "content" : content
        },
        "option": {
        "language": language,
        "model": model,
        "tone": tone,
        "summaryCount" : summaryCount
        }
    }
    # print(json.dumps(data, indent=4, sort_keys=True))
    response = requests.post(url, data=json.dumps(data), headers=headers)
    rescode = response.status_code
    if(rescode == 200):
        r1=response.text  
        # print(r1)

    else:
        print("Error : " + response.text)

#    r1 = json.loads(r1)
#    r1r1= r1['summary']
    r1r1= r1.split('연합뉴스')[1]
    r1r1 = '{}번째 뉴스입니다.{}'.format(i+1, r1r1)
    print(r1r1)
    
    encText = urllib.parse.quote(r1r1)
    data = "speaker=njihun&volume=0&speed=0&pitch=0&format=mp3&text=" + encText;
    url = "https://naveropenapi.apigw.ntruss.com/tts-premium/v1/tts"
    request = urllib.request.Request(url)
    request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
    request.add_header("X-NCP-APIGW-API-KEY",client_secret)
    response = urllib.request.urlopen(request, data=data.encode('utf-8'))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        fn = 'voice{}.mp3'.format(i)
        with open(fn, 'wb') as f:
            f.write(response_body)
        print(fn)
    else:
        print("Error Code:" + rescode)


1번째 뉴스입니다.김부겸 국무총리는 코로나 확산세와 관련해 \"앞으로 1~2주간이 코로나 위기 극복의 전환점이 될 것\"이라며 \"정부는 정점 이후의 상황까지 염두에 두면서 중증과 사망을 최소화하는 데 총력을 기울이겠다\"고 밝혔습니다.\n이어 \"오미크론 하위 변위인 '스텔스 오미크론'의 국내 점유율이 40%를 넘어섰다\"며 다만 \"'스텔스 오미크론'은 오미크론에 비해 전파력이 다소 높을 뿐 중증화율·입원율 등에 차이가 없고 치료제나 백신의 효과도 같다\"고 설명했습니다.\n그러면서 \"질병청이 확진자 643만여 명을 분석한 결과, 미접종자의 중증화율은 3차접종 후 돌파 감염된 확진자에 비해 37배나 높은 것으로 나타났다\"며 백신 접종을 거듭 당부했습니다."}
voice0.mp3
2번째 뉴스입니다.중앙방역대책본부는 오늘 0시 기준 코로나19 신규 확진자가 35만3,980명으로 집계됐다고 밝혔습니다.\n지역별로는 경기 96,257명, 서울 65,033명 등 수도권에서 17만8,859명이 확진됐고, 비수도권에서는 17만5,121명이 양성 판정을 받았습니다.\n위중증 환자는 26명이 줄어 1천104명이고, 어제 하루 코로나로 인한 사망자는 384명으로 코로나19 사태 발생 이후 역대 두번째로 많았습니다."}
voice1.mp3
3번째 뉴스입니다.코로나19 사태 속에서 물가 상승에 따른 부담이상대적으로 저소득층에 집중됐다는 분석이 나왔습니다.\n한경연은 이같은 결과에 대해 저소득층 지출 비중이 높은 품목의 물가가 상대적으로 크게 상승했기 때문이라고 설명했습니다.\n저소득층인 소득 1분위 지출 비중이 5분위보다 높은 <주거·수도·광열>, <식료품>, <보건> 물가는 평균 3.7% 상승한 반면 5분위의 지출 비중이 1분위보다 더 높은 <교육>, <교통>, <음식·숙박> 물가는 평균 0.7% 상승하는 데 그쳤습니다."}
voice2.mp3


### 여기서 고려 사항 1

원래는 json {"summary":"내용"} 형식으로 되어있기에 "내용"을 가져오기 위해

- r1 = json.loads(r1)
- r1r1= r1['summary']

이렇게 진행하는 것이 맞으나,
내용 속 필요없는 "사진 제공:연합뉴스"부분을 제거하기 위해

- r1r1= r1.split('연합뉴스')[1]
- r1r1 = '{}번째 뉴스입니다.{}'.format(i+1, r1r1)

이렇게 진행하게되었다.

## 음성파일 읽어주기!

- playsound

In [12]:
# !pip install playsound
from playsound import playsound

In [13]:
playsound('./voice0.mp3')
playsound('./voice1.mp3')
playsound('./voice2.mp3')

In [21]:
# 파일 하나로 합치기

audio1 = open("voice0.mp3", "rb").read()
audio2 = open("voice1.mp3", "rb").read()
audio3 = open("voice2.mp3", "rb").read()
audioJoin = audio1 + audio2 + audio3
audioFinal = open("audioFinal_2.mp3", "wb").write(audioJoin)

In [ ]:
playsound('./audioFinal_2.mp3')

- pygame

In [26]:
# ! pip3 install pygame
import pygame

pygame.mixer.init()
pygame.mixer.music.load('voice1.mp3')
pygame.mixer.music.play()

In [ ]:
pygame.mixer.init()
pygame.mixer.music.load('voice1.mp3')
pygame.mixer.music.play()
pygame.quit()

In [ ]:
pygame.quit()

In [50]:
#'voice{}.mp3'.format(i)
# len('voice1.mp3')
len(fn)

10

- 포구문으로 돌려보려했는데 잘 안되네,,

In [52]:
for i in range(fn):
    fn = 'voice{}.mp3'.format(i)
    playsound('./voice{}.mp3'.format(i))
    

TypeError: 'str' object cannot be interpreted as an integer

- 요것은 창띄워서 키로 볼륨, 재생, 일시정지 해보기

In [1]:
# import pygame


# 스크린 전체 크기 지정
SCREEN_WIDTH = 400
SCREEN_HEIGHT  = 500
 
pygame.init()

# 스크린 객체 저장
SCREEN = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("news 줄 넘 귀 test")

 
# FPS를 위한 Clock 생성
clock = pygame.time.Clock()

 
# wav, mp3, ogg 가능
pygame.mixer.music.load('voice0.mp3')


# Music stream 무한 반복
pygame.mixer.music.play()

 
playing = True
while playing:
    for event in pygame.event.get():

        if event.type == pygame.QUIT:
            playing = False
            pygame.quit()


        if event.type == pygame.KEYDOWN:

 

            if event.key == pygame.K_UP:
                v = pygame.mixer.music.get_volume()
                pygame.mixer.music.set_volume(v + 0.1)
                print("volume up")

 

            if event.key == pygame.K_DOWN:
                v = pygame.mixer.music.get_volume()
                pygame.mixer.music.set_volume(v - 0.1)
                print("volume down")

 

            if event.key == pygame.K_LEFT:
                pygame.mixer.music.pause()
                print("일시 멈춤")

 

            if event.key == pygame.K_RIGHT:
                pygame.mixer.music.unpause()
                print("다시 재생")
              
            
            if event.key == pygame.K_a:
                pygame.quit()
                print("종료")
                


    # 1초에 60번의 빈도로 순환하기
    # clock.tick(60)

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
volume up
volume up
volume up
volume up
volume up
volume down
volume down
volume down
volume down
volume down
volume down
volume down
일시 멈춤
다시 재생
volume up
volume up
volume up
volume up
volume up
volume up
volume up
종료


error: video system not initialized

In [1]:
# import pygame


# 스크린 전체 크기 지정
SCREEN_WIDTH = 400
SCREEN_HEIGHT  = 500
 
pygame.init()

# 스크린 객체 저장
SCREEN = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("news 줄 넘 귀 test")

 
# FPS를 위한 Clock 생성
clock = pygame.time.Clock()

 
# wav, mp3, ogg 가능
playsound('./voice0.mp3')


# Music stream 무한 반복
#pygame.mixer.music.play()

 
playing = True
while playing:
    for event in playsound.event.get():

        if event.type == playsound.QUIT:
            playing = False
            pygame.quit()


        if event.type == playsound.KEYDOWN:

 

            if event.key == pygame.K_UP:
                v = pygame.mixer.music.get_volume()
                pygame.mixer.music.set_volume(v + 0.1)
                print("volume up")

 

            if event.key == pygame.K_DOWN:
                v = pygame.mixer.music.get_volume()
                pygame.mixer.music.set_volume(v - 0.1)
                print("volume down")

 

            if event.key == pygame.K_LEFT:
                pygame.mixer.music.pause()
                print("일시 멈춤")

 

            if event.key == pygame.K_RIGHT:
                pygame.mixer.music.unpause()
                print("다시 재생")
              
            
            if event.key == pygame.K_a:
                pygame.quit()
                print("종료")
                


    # 1초에 60번의 빈도로 순환하기
    # clock.tick(60)

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
volume up
volume up
volume up
volume up
volume up
volume down
volume down
volume down
volume down
volume down
volume down
volume down
일시 멈춤
다시 재생
volume up
volume up
volume up
volume up
volume up
volume up
volume up
종료


error: video system not initialized